# Astrobrowser - Herschel Reference Survey

Explore the HiPS maps available for the galaxies in the HRS and compute aperture photometry.

# 1. Initialisation

In [ ]:
%matplotlib ipympl
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.table import Table
from astropy import units as u
from scripts import astrobrowser
from photutils.aperture import SkyEllipticalAperture
import importlib

## Read input catalogue

In [ ]:
HRS_catalogue = Table.read(os.path.join('HRS', 'cortese_2014_table2.vot'))

In [ ]:
HRS_catalogue

# 2. Aperture photometry

Consider only the first $n$ galaxies (for debugging)

In [ ]:
n = len(HRS_catalogue)
n = 3
print(f'Considering the first {n} galaxies')

In [ ]:
def find_bg(data):
    p16, p50 = np.nanpercentile(data, [16, 50])
    mu0 = p50
    sigma0 = p50 - p16
    weight = np.exp(-.5 * ((data - mu0) / sigma0)**2)
    total_weight = np.nansum(weight)
    mu1 = np.nansum(weight * data) / total_weight
    sigma1 = np.nansum(weight * data**2) / total_weight
    sigma1 = np.sqrt(sigma1 - mu1**2)
    print(mu0, sigma0)
    print(mu1, sigma1)
    #ivar = 1/sigma1**2 - 1/sigma0**2
    #mu = (mu1/sigma1**2 - mu0/sigma0**2) / ivar
    #print(mu, np.sqrt(1/ivar))
    #return mu, np.sqrt(1/ivar)
    return mu1, sigma1
    

importlib.reload(astrobrowser)
band = 'PACS160'
hips_service_url = 'http://skies.esac.esa.int/Herschel/PACS160'
beam = (2.85 * u.arcsec)**2
skymap_units = u.Jy / beam

pixel_arcsec = 2.
pixel_area = (pixel_arcsec * u.arcsec)**2  # cos(DEC) due to Mercator projection added below
unit_conversion = (skymap_units * pixel_area).to_value(u.Jy)

with PdfPages(os.path.join('HRS', 'output', f'{band}_maps.pdf')) as pdf:
    flux_Jy = []
    flux_err_Jy = []
    for galaxy in HRS_catalogue[:n]:
    
        position = SkyCoord(galaxy['R.A.__J.2000_'], galaxy['Dec__J.2000_'],  unit=(u.hourangle, u.deg))
        radius_arcsec = 2 * galaxy['a']
        print(f"> Downloading HRS-{galaxy['HRS']}:")
        header, data = astrobrowser.get_cutout(hips_service_url, position.ra.deg, position.dec.deg, radius_arcsec, pixel_arcsec)
        if header is None:
            flux_Jy.append(np.nan)
            flux_err_Jy.append(np.nan)
            continue
        wcs = WCS(header)

        aperture = SkyEllipticalAperture(position, a=galaxy['a']*u.arcsec, b=galaxy['b']*u.arcsec, theta=-galaxy['P.A.']*u.deg)  # Why do I have to invert PA?
        pixel_aperture = aperture.to_pixel(wcs)
        pixel_aperture.positions = [np.array(data.shape) / 2]  # dirty fix

        flux = pixel_aperture.do_photometry(data)[0][0]
        mean = flux / pixel_aperture.area
        bg, bg_err = find_bg(data)

        corrected_flux = flux - bg*pixel_aperture.area
        corrected_flux *= unit_conversion * np.cos(position.dec)
        flux_err = bg_err * pixel_aperture.area * unit_conversion * np.cos(position.dec)
        print(f'area = {pixel_aperture.area:.2f}, mean({mean:.3g}) - bg ({bg:.3g}) = {mean-bg:.3g} +- {bg_err:.3g}')
        print(f'flux = {corrected_flux:.3g} +- {flux_err:.3g} ({flux:.3g})')
        print('catalogue', galaxy['F_160'], galaxy['sigma_160'], corrected_flux/galaxy['F_160'])
        flux_Jy.append(corrected_flux)
        flux_err_Jy.append(flux_err)
        
        plt.figure()
        plt.title(f"HRS-{galaxy['HRS']} {band}: ${corrected_flux:.3g} \pm {flux_err:.3g}\ ({galaxy['F_160']:.3g} \pm {galaxy['sigma_160']:.3g})$ Jy")
        im = plt.imshow(data, interpolation='nearest', origin='lower', vmin=bg-bg_err, vmax=mean+bg_err, cmap='terrain')
        plt.contour(data, levels=[mean], colors=['k'])
        pixel_aperture.plot(color='w')
        cb = plt.colorbar(im)
        cb.ax.axhline(mean, c='w')
        cb.ax.axhline(mean, c='k')
        cb.ax.axhline(bg + bg_err, c='w', ls=':')
        cb.ax.axhline(bg, c='w', ls='--')
        cb.ax.axhline(bg - bg_err, c='w', ls=':')
        pdf.savefig()
        plt.close()

In [ ]:
n = len(flux_Jy)
with PdfPages(os.path.join('HRS', 'output', f'{band}_summary.pdf')) as pdf:
    plt.figure()
    plt.errorbar(HRS_catalogue['F_160'][:n], flux_Jy, flux_err_Jy, HRS_catalogue['sigma_160'][:n], fmt='none', alpha=.2)
    plt.grid(alpha=.2)
    x = [np.min(HRS_catalogue['F_160']), np.max(HRS_catalogue['F_160'])]
    plt.plot(x, x, 'k:')
    plt.xscale('log')
    plt.yscale('log')
    plt.ylim(x)
    plt.title(band)
    plt.ylabel('flux measured from HiPS map [Jy]')
    plt.xlabel('HRS flux [Jy]')
    pdf.savefig()
    plt.close()

    plt.figure()
    x = np.array(flux_Jy / HRS_catalogue['F_160'][:n])
    p16, p50, p84 = np.nanpercentile(x.data, [16, 50, 84])
    plt.hist(x, bins=np.logspace(-1, 1, 50), density=False, alpha=.5)
    plt.axvline(p50, c='k', ls='--', label=f'[{p16:.3g}, {p50:.3g}, {p84:.3g}]')
    plt.axvline(p16, c='k', ls=':')
    plt.axvline(p84, c='k', ls=':')
    plt.xscale('log')
    plt.grid(alpha=.2)
    plt.title(band)
    plt.xlabel('HiPS / HRS')
    plt.ylabel('number of galaxies')
    #plt.ylabel('probability')
    plt.legend()
    pdf.savefig()
    plt.close()
    
    plt.figure()
    x = np.array((flux_Jy - HRS_catalogue['F_160'][:n]) / flux_err_Jy)
    mu = np.nanmean(x)
    sigma = np.nanstd(x)
    p16, p50, p84 = np.nanpercentile(x.data, [16, 50, 84])
    plt.axvline(p16, c='b', ls=':')
    plt.axvline(p50, c='b', ls='--')
    plt.axvline(p84, c='b', ls=':')
    plt.hist(x, bins=np.linspace(-5, 5, 50), density=False, label=f'HiPS error: [{p16:.3g}, {p50:.3g}, {p84:.3g}]', alpha=.5)
    x = np.array((flux_Jy - HRS_catalogue['F_160'][:n]) / HRS_catalogue['sigma_160'][:n])
    mu = np.nanmean(x)
    sigma = np.nanstd(x)
    p16, p50, p84 = np.nanpercentile(x.data, [16, 50, 84])
    plt.axvline(p16, c='r', ls=':')
    plt.axvline(p50, c='r', ls='--')
    plt.axvline(p84, c='r', ls=':')
    plt.hist(x, bins=np.linspace(-5, 5, 50), density=False, label=f'HRS error: [{p16:.3g}, {p50:.3g}, {p84:.3g}]', alpha=.5)
    plt.grid(alpha=.2)
    plt.title(band)
    plt.xlabel('(HiPS - HRS) / error')
    plt.ylabel('number of galaxies')
    plt.legend()
    pdf.savefig()
    plt.close()

# -- OLD STUFF --

In [ ]:
raise -1  # STOP HERE

In [ ]:
names = [f'HRS-{ID:03d}' for ID in HRS_catalogue['HRS']]

In [ ]:
positions = SkyCoord(HRS_catalogue['R.A.__J.2000_'], HRS_catalogue['Dec__J.2000_'],  unit=(u.hourangle, u.deg))

In [ ]:
catalogue = Table([names,
                   positions.ra.deg,
                   positions.dec.deg,
                   2 * HRS_catalogue['a'],
                   2.85 * np.ones(len(HRS_catalogue))
                  ], names=['ID', 'RA', 'DEC', 'RADIUS_ARCSEC', 'PIXEL_SIZE_ARCSEC'])

In [ ]:
catalogue

# 2. Explore available skymaps

Select the maps for this galaxy:

In [ ]:
importlib.reload(astrobrowser)
x = astrobrowser.DataExplorer(catalogue, 'ESAVO/P/HERSCHEL/PACS160')

In [ ]:
galaxy = HRS_catalogue[x.galaxy_index]
position = SkyCoord(galaxy['R.A.__J.2000_'], galaxy['Dec__J.2000_'],  unit=(u.hourangle, u.deg))
aperture = SkyEllipticalAperture(position, a=galaxy['a']*u.arcsec, b=galaxy['b']*u.arcsec, theta=-galaxy['P.A.']*u.deg)  # Why do I have to invert PA?
pixel_aperture = aperture.to_pixel(x.wcs)
pixel_aperture.positions = [np.array(x.data.shape) / 2]  # dirty fix
pixel_aperture.plot(x.ax1)

In [ ]:
flux = pixel_aperture.do_photometry(x.data)[0][0]
mean = flux / pixel_aperture.area
source_mask = np.nanmedian(x.data)
source_mask += np.sqrt(np.nanmean((x.data - source_mask)**2))
x.ax1.contour(x.data, levels=[source_mask], colors=['y'])

bg = np.nanmedian(x.data[x.data < source_mask])
#bg_err = np.sqrt(np.nanmean((bg - x.data[x.data < bg])**2))
bg_err = bg - np.nanmedian(x.data[x.data < bg])
x.ax1_cb.axhline(source_mask, c='y')
x.ax1_cb.axhline(bg+bg_err, c='k', ls=':')
x.ax1_cb.axhline(bg, c='k')
x.ax1_cb.axhline(bg-bg_err, c='k', ls=':')

#scale = (3.2/catalogue[x.galaxy_index]['PIXEL_SIZE_ARCSEC'])**2
scale = 1
corrected_flux = flux - bg*pixel_aperture.area
print(f'area = {pixel_aperture.area:.2f}, mean({mean:.3g}) - bg ({bg:.3g}) = {mean-bg:.3g} +- {bg_err:.3g}')
print(f'flux = {corrected_flux:.3g} +- {bg_err*pixel_aperture.area:.3g} ({flux:.3g})')
#print(f'masked = {np.sum(x.data[x.data > source_mask]) - bg*np.count_nonzero(x.data > source_mask):.3g} ({np.sum(x.data[x.data > source_mask]):.3g})')
#print(f'scaled = {corrected_flux*scale:.3g} +- {bg_err*(pixel_aperture.area)*scale:.3g}')
print('catalogue', galaxy['F_160'], galaxy['sigma_160'], corrected_flux*scale/galaxy['F_160'])

In [ ]:
pixel_aperture.area, pixel_aperture.area_overlap(x.data)

In [ ]:
aperture.positions, pixel_aperture.positions*2, x.data.shape

In [ ]:
galaxy

In [ ]:
x.header

In [ ]:
x.header['CDELT1']*3600